In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/kernel-methods-ammi-2023/Xte2.csv
/kaggle/input/kernel-methods-ammi-2023/Xtr0_mat100.csv
/kaggle/input/kernel-methods-ammi-2023/Ytr1.csv
/kaggle/input/kernel-methods-ammi-2023/Xtr0.csv
/kaggle/input/kernel-methods-ammi-2023/Xte1.csv
/kaggle/input/kernel-methods-ammi-2023/Xte0_mat100.csv
/kaggle/input/kernel-methods-ammi-2023/Xtr1_mat100.csv
/kaggle/input/kernel-methods-ammi-2023/Xtr1.csv
/kaggle/input/kernel-methods-ammi-2023/Xtr2.csv
/kaggle/input/kernel-methods-ammi-2023/Xte2_mat100.csv
/kaggle/input/kernel-methods-ammi-2023/Ytr2.csv
/kaggle/input/kernel-methods-ammi-2023/Xtr2_mat100.csv
/kaggle/input/kernel-methods-ammi-2023/Xte0.csv
/kaggle/input/kernel-methods-ammi-2023/Xte1_mat100.csv
/kaggle/input/kernel-methods-ammi-2023/Ytr0.csv


In [2]:
!pip install cvxopt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 47.3 MB/s eta 0:00:0000:0100:01


In [3]:
!pip install cvxpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 45.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.8/298.8 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 68.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 804.0/804.0 kB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 53.1 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 59.8.0
    Uninstalling setuptools-59.8.0:
      Successfully uninstalled setuptools-59.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opentelemetry-api 1.18.0 requires importlib-metadata~=6.0.0, but you have importlib-metadata 6.7.0 which is incompatible.
pymc3 3.11.5 requires numpy<1.22.2,>=1.15.0, but you have numpy 1.23.5 which is incompatible.
pymc3 3.11.5 requ

In [4]:
import numpy as np
import sklearn
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from gensim.models import FastText


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


**1) Let's implement here the Logistic Regression**

In [5]:
import numpy as np


def sigmoid(x):
    return 1 / (1 + np.exp(-x))


class LogisticRegression():
    """
    Logistic Regression implementation
    Usage:
    """

    def __init__(self, kernel, lambda_=1.):
        """
        lambda_: float, regularization parameter
        """
        self.kernel = kernel
        self.lambda_ = lambda_

    def fit(self, X, y):
        """ Solve KLR using
        X: array (n_samples, n_features) \\
        y: array of -1 or 1 (n_samples,1)
        """

        self.X_train = X
        n_samples = X.shape[0]
        num_iter = 10
        eps = 1e-6
        K = self.kernel.gram(X)

        # Initialization
        alpha = np.zeros((n_samples, 1))

        for i in range(num_iter):
            alpha_old = alpha

            m = K @ alpha
            W = sigmoid(m) * sigmoid(-m)
            z = m + y / sigmoid(-y * m)

            # Solve WKRR
            sqrt_W = np.sqrt(W)
            alpha = sqrt_W * np.linalg.solve(sqrt_W * K * sqrt_W.T + n_samples * self.lambda_ * np.eye(n_samples),
                                             sqrt_W * y)

            if np.sum((alpha - alpha_old) ** 2) < eps:
                break

        self.alphas = alpha
        return self.alphas

    def predict(self, X):
        """
        X: array (n_samples, n_features)\\
        Return: float array (n_samples,)
        """
        K = self.kernel.gram(X, self.X_train)
        y = np.dot(K, self.alpha)
        return y

    def predict_classes(self, X, threshold=0):
        """
        X: array (n_samples, n_features)\\
        Return: 0 and 1 array (n_samples,)
        """
        K = self.kernel.gram(X, self.X_train)
        y = np.dot(K, self.alpha)
        return np.where(y > threshold, 1, 0)

**2) Ridge Regression**

In [6]:
import numpy as np
import cvxopt

class RidgeRegression():
    """
    Ridge Regression implementation

    Usage:
    """

    def __init__(self, kernel, alpha):
        """
        alpha: float > 0, default=1.0
            Regularization strength
        """
        self.kernel = kernel
        self.alpha = alpha

    def fit(self, X, y):
        """ Solve KRR using alpha = (K + lambda n I)^(-1) y \\
        X: array (n_samples, n_features) \\
        y: array of 0 or 1 (n_samples,)   
        """
        self.X_train = X
        n_samples = X.shape[0]
        K = self.kernel.gram(X)
        self.alphas = np.linalg.solve(K+n_samples*self.alpha*np.eye(n_samples), y)
        return self.alphas

    def predict(self, X):
        """
        X: array (n_samples, n_features)\\
        Return: float array (n_samples,)
        """
        K = self.kernel.gram(X, self.X_train)
        y = np.dot(K, self.alphas)
        return y

    def predict_classes(self, X, threshold=0.5):
        """ 
        X: array (n_samples, n_features)\\
        Return: 0 and 1 array (n_samples,)
        """
        K = self.kernel.gram(X, self.X_train)
        y = np.dot(K, self.alphas)
        return np.where(y>threshold, 1, 0)

**3) SVM**

In [7]:
import numpy as np
import cvxopt
from cvxopt import matrix
import cvxpy as cp

class SVM():
    """
    SVM implementation
    
    Usage:
        svm = SVM(kernel='linear', C=1)
        svm.fit(X_train, y_train)
        svm.predict(X_test)
    """

    def __init__(self, kernel, C=1.0, tol_support_vectors=1e-4):
        """
        kernel: Which kernel to use
        C: float > 0, default=1.0, regularization parameter
        tol_support_vectors: Threshold for alpha value to consider vectors as support vectors
        """
        self.kernel = kernel
        self.C = C
        self.tol_support_vectors = tol_support_vectors

    def fit(self, X, y):

        self.X_train = X
        n_samples = X.shape[0]
        print("Computing the kernel...")
        self.X_train_gram = self.kernel.gram(X)
        print("Done!")

        #Define the optimization problem to solve

        P = self.X_train_gram
        q = -y.astype('float')
        G = np.block([[np.diag(np.squeeze(y).astype('float'))],[-np.diag(np.squeeze(y).astype('float'))]])
        h = np.concatenate((self.C*np.ones(n_samples),np.zeros(n_samples)))

        #Solve the problem
        #With cvxopt

        P=matrix(P)
        q=matrix(q)
        G=matrix(G)
        h=matrix(h)
        solver = cvxopt.solvers.qp(P=P,q=q,G=G,h=h)
        x = solver['x']
        self.alphas = np.squeeze(np.array(x))

        #Retrieve the support vectors
        self.support_vectors_indices = np.squeeze(np.abs(np.array(x))) > self.tol_support_vectors
        self.alphas = self.alphas[self.support_vectors_indices]
        self.support_vectors = self.X_train[self.support_vectors_indices]

        print(len(self.support_vectors), "support vectors out of",len(self.X_train), "training samples")

        return self.alphas


    def predict(self, X):
        """
        X: array (n_samples, n_features)\\
        Return: float array (n_samples,)
        """
        K = self.kernel.gram(X, self.support_vectors)
        y = np.dot(K, self.alphas)
        return y

    def predict_classes(self, X, threshold=0):
        """
        X: array (n_samples, n_features)\\
        Return: 0 and 1 array (n_samples,)
        """
        K = self.kernel.gram(X, self.support_vectors)
        y = np.dot(K, self.alphas)
        return np.where(y > threshold, 1, -1)

**4) All Kernels functions**

In [8]:
import numpy as np
import scipy.sparse as sparse
from tqdm import tqdm

class Kernel():
    """ Abstract Kernel class"""

    def __init__(self):
        pass

    def similarity(self, x, y):
        """ Similarity between 2 feature vectors (depends on the type of kernel)"""
        return -1

    def gram(self, X1, X2=None):
        """ Compute the gram matrix of a data vector X where the (i,j) entry is defined as <Xi,Xj>\\
        X1: data vector (n_samples_1 x n_features)
        X2: data vector (n_samples_2 x n_features), if None compute the gram matrix for (X1,X1)
        """
        if X2 is None: 
            X2=X1
        n_samples_1 = X1.shape[0]
        n_samples_2 = X2.shape[0]
        G = np.zeros((n_samples_1, n_samples_2))
        for ii in tqdm(range(n_samples_1)):
            for jj in range(n_samples_2):
                G[ii,jj] = self.similarity(X1[ii], X2[jj])
        return G


class SumKernel(Kernel):

    def __init__(self, kernels, weights=None):
        """ kernels: list of kernels """
        self.kernels = kernels
        self.weights = weights
        if self.weights is None:
            self.weights = [1.0 for _ in kernels]
        super().__init__()

    def similarity(self, x, y):
        """ x, y: string """
        s = self.kernels[0].similarity(x,y) * self.weights[0]
        for ii, kernel in enumerate(self.kernels[1:]):
            s += kernel.similarity(x,y) * self.weights[ii]
        return s

    def gram(self, X1, X2=None):
        """ Compute the sum of the gram matrices of all kernels\\
        X1: array of string (n_samples_1,)
        X2: array of string (n_samples_2,), if None compute the gram matrix for (X1,X1)
        """
        G = self.kernels[0].gram(X1,X2) * self.weights[0]
        for ii, kernel in tqdm(enumerate(self.kernels[1:])):
            G += kernel.gram(X1,X2) * self.weights[ii]
        return G


class LinearKernel(Kernel):

    def __init__(self):
        super().__init__()

    def similarity(self, x, y):
        """ linear kernel : k(x,y) = <x,y> \\
        x, y: array (n_features,)
        """
        return np.dot(x,y)


class GaussianKernel(Kernel):

    def __init__(self, sigma,normalize=True):
        super().__init__()
        self.sigma = sigma
        self.normalize = normalize

    def similarity(self, x, y):
        """ gaussian kernel : k(x,y) = 1/ sqrt(2 pi sigma2)^n * exp( - ||x-y||^2 / 2 sigma^2 )\\
        x, y: array (n_features,)
        """

        if self.normalize:
            norm_fact = (np.sqrt(2 * np.pi) * self.sigma) ** len(x)
            return np.exp(-np.linalg.norm(x-y)**2 / (2 * self.sigma**2)) / norm_fact
        else:
            return np.exp(-np.linalg.norm(x-y)**2 / (2 * self.sigma**2))


class PolynomialKernel(Kernel):

    def __init__(self, gamma=1, coef0=1, degree=3):
        super().__init__()
        self.gamma = gamma
        self.coef0 = coef0
        self.degree = degree

    def similarity(self, x, y):
        """ polynomial kernel : k(x,y) = (gamma <x,y> + r)^d \\
        x, y: array (n_features,)
        """
        return (self.gamma * np.dot(x,y) + self.coef0)**self.degree


class SpectrumKernel(Kernel):

    def __init__(self, k):
        super().__init__()
        self.k = k

    def similarity(self, x, y):
        """ Spectrum kernel \\
        x, y: string
        """
        substr_x, counts_x = np.unique([x[i:i+self.k] for i in range(len(x)-self.k+1)], return_counts=True)
        return np.sum(np.char.count(y, substr_x)*counts_x)


class MismatchKernel(Kernel):

    def __init__(self, k, m, neighbours, kmer_set, normalize=False):
        super().__init__()
        self.k = k
        self.m = m
        self.kmer_set = kmer_set #kmer_set and neighbours have to be pre-computed (to save computational time when running multiple experiments)
        self.neighbours = neighbours
        self.normalize = normalize

    def neighbour_embed_kmer(self, x):
        """
        Embed kmer with neighbours.
        x: str
        """
        kmer_x = [x[j:j + self.k] for j in range(len(x) - self.k + 1)]
        x_emb = {}
        for kmer in kmer_x:
            neigh_kmer = self.neighbours[kmer]
            for neigh in neigh_kmer:
                idx_neigh = self.kmer_set[neigh]
                if idx_neigh in x_emb:
                    x_emb[idx_neigh] += 1
                else:
                    x_emb[idx_neigh] = 1
        return x_emb
        

    def neighbour_embed_data(self, X):
        """
        Embed data with neighbours.
        X: array of string
        """
        X_emb = []
        for i in range(len(X)):
            x = X[i]
            x_emb = self.neighbour_embed_kmer(x)
            X_emb.append(x_emb)
        return X_emb
    
    def to_sparse(self, X_emb):
        """
        Embed data to sparse matrix.
        X_emb: list of dict.
        """
        data, row, col = [], [], []
        for i in range(len(X_emb)):
            x = X_emb[i]
            data += list(x.values())
            row += list(x.keys())
            col += [i for j in range(len(x))]
        X_sm = sparse.coo_matrix((data, (row, col)))
        return X_sm

    def similarity(self, x, y):
        """ Mismatch kernel \\
        x, y: string
        """
        x_emb = self.neighbour_embed_kmer(x)
        y_emb = self.neighbour_embed_kmer(y)
        sp = 0
        for idx_neigh in x_emb:
            if idx_neigh in y_emb:
                sp += x_emb[idx_neigh] * y_emb[idx_neigh]
        if self.normalize:
            sp /= np.sqrt(np.sum(np.array(list(x_emb.values()))**2))
            sp /= np.sqrt(np.sum(np.array(list(y_emb.values()))**2))
        return sp

    def gram(self, X1, X2=None):
        """ Compute the gram matrix of a data vector X where the (i,j) entry is defined as <Xi,Xj>\\
        X1: array of string (n_samples_1,)
        X2: array of string (n_samples_2,), if None compute the gram matrix for (X1,X1)
        """
        
        X1_emb = self.neighbour_embed_data(X1)
        X1_sm = self.to_sparse(X1_emb)
        
        if X2 is None:
            X2 = X1
        X2_emb = self.neighbour_embed_data(X2)
        X2_sm = self.to_sparse(X2_emb)

        # Reshape matrices if the sizes are different
        nadd_row = abs(X1_sm.shape[0] - X2_sm.shape[0])
        if X1_sm.shape[0] > X2_sm.shape[0]:
            add_row = sparse.coo_matrix(([0], ([nadd_row-1], [X2_sm.shape[1]-1])))
            X2_sm = sparse.vstack((X2_sm, add_row))
        elif X1_sm.shape[0] < X2_sm.shape[0]:
            add_row = sparse.coo_matrix(([0], ([nadd_row - 1], [X1_sm.shape[1] - 1])))
            X1_sm = sparse.vstack((X1_sm, add_row))

        G = (X1_sm.T * X2_sm).todense().astype('float')
        
        if self.normalize:
            G /= np.array(np.sqrt(X1_sm.power(2).sum(0)))[0,:,None]
            G /= np.array(np.sqrt(X2_sm.power(2).sum(0)))[0,None,:]
            
        return G

**5) Utils functions to compute Mismatch**

**6) Data and Pre-processing**

In [9]:
# Imports
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm

# Read csv files

# shape (2000,1): string
X0_train = pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Xtr0.csv", sep=",", index_col=0).values
X1_train = pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Xtr1.csv", sep=",", index_col=0).values
X2_train = pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Xtr2.csv", sep=",", index_col=0).values

# shape (2000,100): float
X0_mat100_train = pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Xtr0_mat100.csv", sep=" ", header=None).values
X1_mat100_train = pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Xtr1_mat100.csv", sep=" ", header=None).values
X2_mat100_train = pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Xtr2_mat100.csv", sep=" ", header=None).values

# shape (2000,1): string
X0_test = pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Xte0.csv", sep=",", index_col=0).values
X1_test = pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Xte1.csv", sep=",", index_col=0).values
X2_test = pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Xte2.csv", sep=",", index_col=0).values

# shape (2000,100): float
X0_mat100_test = pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Xte0_mat100.csv", sep=" ", header=None).values
X1_mat100_test = pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Xte1_mat100.csv", sep=" ", header=None).values
X2_mat100_test = pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Xte2_mat100.csv", sep=" ", header=None).values

# shape (2000,1): 0 or 1
Y0_train = pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Ytr0.csv", sep=",", index_col=0).values
Y1_train = pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Ytr1.csv", sep=",", index_col=0).values
Y2_train = pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Ytr2.csv", sep=",", index_col=0).values


#Put test matrices into the right format
X0_train = X0_train[:,0]
X1_train = X1_train[:,0]
X2_train = X2_train[:,0]

X0_test = X0_test[:,0]
X1_test = X1_test[:,0]
X2_test = X2_test[:,0]

#Rescaling labels
Y0_train = np.where(Y0_train == 0, -1, 1)
Y1_train = np.where(Y1_train == 0, -1, 1)
Y2_train = np.where(Y2_train == 0, -1, 1)



C = 1.0
#gamma = 1/(X_mat_train.shape[1] * X_mat_train.var())
coef0 = 1.0
degree = 3
k = 12

In [10]:
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm


def create_kmer_set(X, k, kmer_set={}):
    """
    Return a set of all kmers appearing in the dataset.
    """
    len_seq = len(X[0])
    idx = len(kmer_set)
    for i in range(len(X)):
        x = X[i]
        kmer_x = [x[i:i + k] for i in range(len_seq - k + 1)]
        for kmer in kmer_x:
            if kmer not in kmer_set:
                kmer_set[kmer] = idx
                idx += 1
    return kmer_set


def m_neighbours(kmer, m, recurs=0):
    """
    Return a list of neighbours kmers (up to m mismatches).
    """
    if m == 0:
        return [kmer]

    letters = ['G', 'T', 'A', 'C']
    k = len(kmer)
    neighbours = m_neighbours(kmer, m - 1, recurs + 1)

    for j in range(len(neighbours)):
        neighbour = neighbours[j]
        for i in range(recurs, k - m + 1):
            for l in letters:
                neighbours.append(neighbour[:i] + l + neighbour[i + 1:])
    return list(set(neighbours))


def get_neighbours(kmer_set, m):
    """
    Find the neighbours given a set of kmers.
    """
    kmers_list = list(kmer_set.keys())
    kmers = np.array(list(map(list, kmers_list)))
    num_kmers, kmax = kmers.shape
    neighbours = {}
    for i in range(num_kmers):
        neighbours[kmers_list[i]] = []

    for i in tqdm(range(num_kmers)):
        kmer = kmers_list[i]
        kmer_neighbours = m_neighbours(kmer, m)
        for neighbour in kmer_neighbours:
            if neighbour in kmer_set:
                neighbours[kmer].append(neighbour)
    return neighbours


def load_neighbors(dataset, k, m):
    """
    dataset: 0, 1 or 2\\
    k: len of the kmers
    m: number of possible mismatches
    """
    directory = 'saved1_neighbors/'
    if not os.path.exists(directory):
        os.makedirs(directory)
        
    file_name = 'neighbours_'+str(dataset)+'_'+str(k)+'_'+str(m)+'.p'
    
                  
    # Load
    neighbours, kmer_set = pickle.load(open('saved_neighbors/'+file_name, 'rb'))
    print('Neighbors correctly loaded!')
    return neighbours, kmer_set


def load_or_compute_neighbors(dataset,k,m):
    """
    dataset: 0, 1 or 2\\
    k: len of the kmers
    m: number of possible mismatches
    """
    
    try:
        #Load the neighbors
        neighbours, kmer_set = load_neighbors(dataset, k, m)
    except:
        print('No file found, creating kmers neighbors')
        #Compute the neighbors
        directory = 'saved_neighbors/'
        if not os.path.exists(directory):
            os.makedirs(directory)
        file_name = 'neighbours_'+str(dataset)+'_'+str(k)+'_'+str(m)+'.p'
        if dataset==0:
            X0_train = pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Xtr0.csv", sep=",", index_col=0).values
            X0_test =  pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Xte0.csv", sep=",", index_col=0).values
            kmer_set = create_kmer_set(X0_train[:,0], k, kmer_set={})
            kmer_set = create_kmer_set(X0_test[:,0], k, kmer_set)
            neighbours = get_neighbours(kmer_set, m)
            pickle.dump([neighbours, kmer_set], open('saved_neighbors/'+file_name, 'wb'))
        elif dataset==1:
            X1_train = pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Xtr1.csv", sep=",", index_col=0).values
            X1_test =  pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Xte1.csv", sep=",", index_col=0).values
            kmer_set = create_kmer_set(X1_train[:,0], k, kmer_set={})
            kmer_set = create_kmer_set(X1_test[:,0], k, kmer_set)
            neighbours = get_neighbours(kmer_set, m)
            pickle.dump([neighbours, kmer_set], open('saved_neighbors/'+file_name, 'wb'))
        elif dataset==2:
            X2_train = pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Xtr2.csv", sep=",", index_col=0).values
            X2_test = pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Xte2.csv", sep=",", index_col=0).values
            kmer_set = create_kmer_set(X2_train[:,0], k, kmer_set={})
            kmer_set = create_kmer_set(X2_test[:,0], k, kmer_set)
            neighbours = get_neighbours(kmer_set, m)
            pickle.dump([neighbours, kmer_set], open('saved_neighbors/'+file_name, 'wb'))
            
    return neighbours, kmer_set

**II) TEST OF ALL THE METHODS**

**a) The Spectrum kernel**

In [ ]:
# C = 1.0
# #gamma = 1/(X_mat_train.shape[1] * X_mat_train.var())
# coef0 = 1.0
# degree = 3
# k = 12

In [12]:
# Spectrum kernel on dataset_0
our_svm = SVM(kernel=SpectrumKernel(k=k),C=C)   
our_svm.fit(X0_train, Y0_train)
pred0 = our_svm.predict_classes(X0_test)

Computing the kernel...


100%|██████████| 2000/2000 [09:22<00:00,  3.56it/s]


Done!
     pcost       dcost       gap    pres   dres
 0: -9.2515e-01 -2.1339e+03  8e+03  1e+00  1e-15
 1:  1.5905e+01 -6.9684e+02  9e+02  6e-02  6e-16
 2:  1.3552e+00 -1.0194e+02  1e+02  4e-03  6e-16
 3: -9.6108e+00 -1.7246e+01  8e+00  5e-05  7e-16
 4: -1.0325e+01 -1.1055e+01  7e-01  4e-06  3e-16
 5: -1.0344e+01 -1.0395e+01  5e-02  2e-07  2e-16
 6: -1.0346e+01 -1.0347e+01  2e-03  6e-09  2e-16
 7: -1.0346e+01 -1.0346e+01  4e-05  1e-10  2e-16
 8: -1.0346e+01 -1.0346e+01  1e-06  1e-12  2e-16
Optimal solution found.
1941 support vectors out of 2000 training samples


100%|██████████| 1000/1000 [04:31<00:00,  3.68it/s]


In [15]:
#Spectrum kernel on dataset_1
our_svm = SVM(kernel=SpectrumKernel(k=k),C=C)   
our_svm.fit(X1_train, Y1_train)
pred1 = our_svm.predict_classes(X1_test)

Computing the kernel...


100%|██████████| 2000/2000 [09:28<00:00,  3.52it/s]


Done!
     pcost       dcost       gap    pres   dres
 0: -9.0330e-01 -2.1081e+03  8e+03  1e+00  1e-15
 1:  1.6961e+01 -6.4975e+02  8e+02  3e-02  5e-16
 2: -5.3148e+00 -4.0342e+01  4e+01  5e-04  7e-16
 3: -1.0612e+01 -1.3399e+01  3e+00  1e-06  4e-16
 4: -1.0750e+01 -1.0841e+01  9e-02  3e-08  2e-16
 5: -1.0750e+01 -1.0755e+01  5e-03  1e-09  1e-16
 6: -1.0750e+01 -1.0750e+01  1e-04  3e-11  1e-16
 7: -1.0750e+01 -1.0750e+01  1e-06  3e-13  1e-16
Optimal solution found.
1999 support vectors out of 2000 training samples


100%|██████████| 1000/1000 [04:38<00:00,  3.59it/s]


In [20]:
#Spectrum kernel on dataset_2
our_svm = SVM(kernel=SpectrumKernel(k=k),C=C)   
our_svm.fit(X2_train, Y2_train)
pred2 = our_svm.predict_classes(X2_test)

Computing the kernel...


100%|██████████| 2000/2000 [09:20<00:00,  3.57it/s]


Done!
     pcost       dcost       gap    pres   dres
 0: -8.6588e-01 -2.1565e+03  9e+03  1e+00  1e-15
 1:  1.4848e+01 -7.2969e+02  1e+03  8e-02  6e-16
 2:  4.4215e+00 -1.3016e+02  1e+02  7e-03  7e-16
 3: -8.3590e+00 -1.9190e+01  1e+01  1e-04  1e-15
 4: -9.5922e+00 -1.0636e+01  1e+00  7e-06  3e-16
 5: -9.6303e+00 -9.7593e+00  1e-01  8e-07  2e-16
 6: -9.6351e+00 -9.6488e+00  1e-02  7e-08  2e-16
 7: -9.6359e+00 -9.6366e+00  6e-04  3e-09  2e-16
 8: -9.6360e+00 -9.6360e+00  2e-05  7e-11  2e-16
 9: -9.6360e+00 -9.6360e+00  8e-07  1e-12  2e-16
Optimal solution found.
1890 support vectors out of 2000 training samples


100%|██████████| 1000/1000 [04:23<00:00,  3.80it/s]


In [21]:
##### CREATE Spectrum kernel SUBMISSION FILE ####

pred = np.concatenate([pred0.squeeze(),pred1.squeeze(),pred2.squeeze()])
pred = np.where(pred == -1, 0, 1)
pred_df = pd.DataFrame()
print(pred.shape)
print(pred)
pred_df['Bound'] = pred
pred_df.index.name = 'Id'

(3000,)
[1 0 0 ... 0 0 1]


In [22]:
pred_df.to_csv('DD_SPECT1.csv', sep=',', header=True)

#Public score of spectrum: 0.62133

**b) Mismatch Kernel**

In [22]:
X0_train.shape

(2000,)

In [11]:
##### PARAMETERS #####

#kernel = 'sum' #'linear' 'rbf', 'poly', 'spectrum', 'mismatch' or 'sum'
C = 1.0 #Parameter C for SVM
gamma = 10.0 #Parameter gamma for SVM (only for 'rbf' or 'poly')
coef0 = 1.0 #Parameter coef0 for SVM (only for 'poly')
degree = 3 #Parameter degree for SVM (only for 'poly')
k = 12 #Parameter k for SVM (only for 'spectrum' and 'mismatch')
m = 2 #Parameter m for SVM (only for 'mismatch')
list_k = [5,8,10,12,13,15] #List of parameters k for sum of mismatch kernels (only for 'sum')
list_m = [1,1,1,2,2,3] #List of parameters m for sum of mismatch kernels (only for 'sum')
weights = [1.0,1.0,1.0,1.0,1.0,1.0] #List of weights for sum of mismatch kernels (only for 'sum')

In [ ]:
#Generation of the different neighbours and kmer on the data_0_1_2
neighbours_0, kmer_set_0 = load_or_compute_neighbors(0,k,m)
neighbours_1, kmer_set_1 = load_or_compute_neighbors(1,k,m)
neighbours_2, kmer_set_2 = load_or_compute_neighbors(2,k,m)

In [33]:
#Mismatch kernel on dataset_0
our_svm = SVM(kernel=MismatchKernel(k=k, m=m, neighbours=neighbours_0, kmer_set=kmer_set_0,normalize=True), C=C)  
our_svm.fit(X0_train, Y0_train)
pred0M = our_svm.predict_classes(X0_test)

Computing the kernel...
Done!
     pcost       dcost       gap    pres   dres
 0: -7.7115e+02 -4.9180e+03  2e+04  2e+00  2e-15
 1: -6.9341e+02 -3.2455e+03  3e+03  5e-02  8e-16
 2: -7.5458e+02 -1.1223e+03  4e+02  7e-03  8e-16
 3: -8.1589e+02 -9.0818e+02  9e+01  1e-03  7e-16
 4: -8.3455e+02 -8.5915e+02  2e+01  2e-04  6e-16
 5: -8.4022e+02 -8.4388e+02  4e+00  2e-05  7e-16
 6: -8.4117e+02 -8.4141e+02  2e-01  7e-07  7e-16
 7: -8.4124e+02 -8.4125e+02  1e-02  2e-08  8e-16
 8: -8.4125e+02 -8.4125e+02  4e-04  2e-10  7e-16
Optimal solution found.
1806 support vectors out of 2000 training samples


In [37]:
pred0M.shape

(1, 1000)

In [38]:
#Mismatch kernel on dataset_1
our_svm = SVM(kernel=MismatchKernel(k=k, m=m, neighbours=neighbours_1, kmer_set=kmer_set_1,normalize=True), C=C)  
our_svm.fit(X1_train, Y1_train)
pred1M = our_svm.predict_classes(X1_test)

Computing the kernel...
Done!
     pcost       dcost       gap    pres   dres
 0: -8.3621e+02 -4.5428e+03  1e+04  2e+00  2e-15
 1: -7.8430e+02 -2.8610e+03  2e+03  3e-16  6e-16
 2: -8.5461e+02 -1.1029e+03  2e+02  1e-16  7e-16
 3: -9.1228e+02 -9.5664e+02  4e+01  1e-16  7e-16
 4: -9.2476e+02 -9.2888e+02  4e+00  2e-16  7e-16
 5: -9.2596e+02 -9.2661e+02  7e-01  2e-16  6e-16
 6: -9.2617e+02 -9.2621e+02  4e-02  2e-16  7e-16
 7: -9.2619e+02 -9.2619e+02  1e-03  2e-16  6e-16
 8: -9.2619e+02 -9.2619e+02  7e-05  2e-16  7e-16
Optimal solution found.
1963 support vectors out of 2000 training samples


In [39]:
#Mismatch kernel on dataset_2
our_svm = SVM(kernel=MismatchKernel(k=k, m=m, neighbours=neighbours_2, kmer_set=kmer_set_2,normalize=True), C=C)  
our_svm.fit(X2_train, Y2_train)
pred2M = our_svm.predict_classes(X2_test)

Computing the kernel...
Done!
     pcost       dcost       gap    pres   dres
 0: -6.4720e+02 -4.6637e+03  2e+04  2e+00  2e-15
 1: -5.7856e+02 -3.0228e+03  3e+03  1e-01  8e-16
 2: -6.3149e+02 -9.7673e+02  4e+02  1e-02  8e-16
 3: -6.8248e+02 -7.8570e+02  1e+02  3e-03  6e-16
 4: -7.0150e+02 -7.2719e+02  3e+01  3e-04  6e-16
 5: -7.0612e+02 -7.1238e+02  6e+00  5e-05  6e-16
 6: -7.0746e+02 -7.0819e+02  7e-01  3e-06  7e-16
 7: -7.0765e+02 -7.0768e+02  3e-02  4e-08  6e-16
 8: -7.0766e+02 -7.0766e+02  3e-03  4e-09  6e-16
 9: -7.0766e+02 -7.0766e+02  1e-04  4e-11  7e-16
Optimal solution found.
1629 support vectors out of 2000 training samples


In [40]:
##### CREATE MISMATCH SUBMISSION FILE ####

pred = np.concatenate([pred0M.squeeze(),pred1M.squeeze(),pred2M.squeeze()])
pred = np.where(pred == -1, 0, 1)
pred_df = pd.DataFrame()
print(pred.shape)
print(pred)
pred_df['Bound'] = pred
pred_df.index.name = 'Id'

(3000,)
[1 1 0 ... 0 0 1]


In [41]:
pred_df.to_csv('DD_MISMATCH1.csv', sep=',', header=True)

#Public score of Mismatch: 0.67933

**d) Sum of Mismatch Kernel**

In [11]:
C = 1.0 #Parameter C for SVM
gamma = 10.0 #Parameter gamma for SVM (only for 'rbf' or 'poly')
coef0 = 1.0 #Parameter coef0 for SVM (only for 'poly')
degree = 3 #Parameter degree for SVM (only for 'poly')
k = 12 #Parameter k for SVM (only for 'spectrum' and 'mismatch')
m = 2 #Parameter m for SVM (only for 'mismatch')
list_k = [5,8,10,12,13,15] #List of parameters k for sum of mismatch kernels (only for 'sum')
list_m = [1,1,1,2,2,3] #List of parameters m for sum of mismatch kernels (only for 'sum')
weights = [1.0,1.0,1.0,1.0,1.0,1.0] #List of weights for sum of mismatch kernels (only for 'sum')


In [12]:
#Sum kernel on dataset 0

dataset_nbr = 0
kernels = []
for k,m in zip(list_k,list_m):
    neighbours, kmer_set = load_or_compute_neighbors(dataset_nbr, k, m)
    kernels.append(MismatchKernel(k=k, m=m, neighbours=neighbours, kmer_set=kmer_set, normalize = True))
svm = SVM(kernel=SumKernel(kernels=kernels, weights=weights), C=C)
svm.fit(X0_train, Y0_train)
pred0S = svm.predict_classes(X0_test)

No file found, creating kmers neighbors


100%|██████████| 1024/1024 [00:00<00:00, 42370.89it/s]

No file found, creating kmers neighbors



100%|██████████| 53692/53692 [00:02<00:00, 24362.68it/s]


No file found, creating kmers neighbors


...

 84%|████████▍ | 202527/240751 [2:30:29<28:11, 22.60it/s]  IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|██████████| 240751/240751 [2:58:55<00:00, 22.43it/s]


Computing the kernel...


5it [01:08, 13.64s/it]


Done!
     pcost       dcost       gap    pres   dres
 0: -1.6724e+02 -3.2097e+03  1e+04  2e+00  8e-15
 1: -1.3788e+02 -1.7285e+03  2e+03  1e-01  6e-15
 2: -1.7946e+02 -4.1078e+02  2e+02  1e-02  6e-15
 3: -1.9308e+02 -2.5569e+02  6e+01  3e-03  5e-15
 4: -1.9570e+02 -2.0548e+02  1e+01  1e-04  5e-15
 5: -1.9610e+02 -1.9716e+02  1e+00  1e-05  5e-15
 6: -1.9616e+02 -1.9620e+02  3e-02  3e-07  5e-15
 7: -1.9617e+02 -1.9617e+02  9e-04  4e-09  5e-15
 8: -1.9617e+02 -1.9617e+02  3e-05  7e-11  5e-15
Optimal solution found.
1850 support vectors out of 2000 training samples


5it [00:43,  8.73s/it]


In [13]:
#Sum kernel on dataset 1

dataset_nbr = 1
kernels = []
for k,m in zip(list_k,list_m):
    neighbours, kmer_set = load_or_compute_neighbors(dataset_nbr, k, m)
    kernels.append(MismatchKernel(k=k, m=m, neighbours=neighbours, kmer_set=kmer_set, normalize = True))
svm = SVM(kernel=SumKernel(kernels=kernels, weights=weights), C=C)
svm.fit(X1_train, Y1_train)
pred1S = svm.predict_classes(X1_test)

No file found, creating kmers neighbors


100%|██████████| 1024/1024 [00:00<00:00, 41698.31it/s]

No file found, creating kmers neighbors



100%|██████████| 52479/52479 [00:02<00:00, 24720.23it/s]


No file found, creating kmers neighbors


...

100%|██████████| 252237/252237 [3:04:39<00:00, 22.77it/s]  


Computing the kernel...


5it [01:05, 13.14s/it]


Done!
     pcost       dcost       gap    pres   dres
 0: -1.5483e+02 -3.1923e+03  1e+04  2e+00  7e-15
 1: -1.2293e+02 -1.6563e+03  2e+03  6e-02  7e-15
 2: -1.7518e+02 -4.3345e+02  3e+02  8e-03  6e-15
 3: -1.8746e+02 -2.2633e+02  4e+01  1e-03  5e-15
 4: -1.8841e+02 -1.9271e+02  4e+00  6e-05  5e-15
 5: -1.8851e+02 -1.8868e+02  2e-01  2e-06  5e-15
 6: -1.8852e+02 -1.8852e+02  4e-03  3e-08  5e-15
 7: -1.8852e+02 -1.8852e+02  8e-05  4e-10  5e-15
Optimal solution found.
1952 support vectors out of 2000 training samples


5it [00:45,  9.01s/it]


In [14]:
#Sum kernel on dataset 2

dataset_nbr = 2
kernels = []
for k,m in zip(list_k,list_m):
    neighbours, kmer_set = load_or_compute_neighbors(dataset_nbr, k, m)
    kernels.append(MismatchKernel(k=k, m=m, neighbours=neighbours, kmer_set=kmer_set, normalize = True))
svm = SVM(kernel=SumKernel(kernels=kernels, weights=weights), C=C)
svm.fit(X2_train, Y2_train)
pred2S = svm.predict_classes(X2_test)

No file found, creating kmers neighbors


100%|██████████| 1024/1024 [00:00<00:00, 44275.73it/s]

No file found, creating kmers neighbors



100%|██████████| 52569/52569 [00:02<00:00, 25539.21it/s]


No file found, creating kmers neighbors


...

100%|██████████| 230332/230332 [2:51:06<00:00, 22.44it/s]  


Computing the kernel...


5it [01:11, 14.39s/it]


Done!
     pcost       dcost       gap    pres   dres
 0: -1.2649e+02 -3.3363e+03  1e+04  2e+00  7e-15
 1: -9.3519e+01 -1.8732e+03  2e+03  2e-01  5e-15
 2: -1.2669e+02 -3.2560e+02  2e+02  6e-03  6e-15
 3: -1.4654e+02 -1.9389e+02  5e+01  1e-03  5e-15
 4: -1.4967e+02 -1.5995e+02  1e+01  8e-05  4e-15
 5: -1.5029e+02 -1.5225e+02  2e+00  1e-05  4e-15
 6: -1.5045e+02 -1.5068e+02  2e-01  6e-07  4e-15
 7: -1.5047e+02 -1.5051e+02  3e-02  8e-08  4e-15
 8: -1.5048e+02 -1.5048e+02  4e-03  1e-08  4e-15
 9: -1.5048e+02 -1.5048e+02  7e-05  1e-10  5e-15
Optimal solution found.
1696 support vectors out of 2000 training samples


5it [00:43,  8.78s/it]


In [15]:
##### CREATE SUM MISMATCH KERNEL SUBMISSION FILE ####

pred = np.concatenate([pred0S.squeeze(),pred1S.squeeze(),pred2S.squeeze()])
pred = np.where(pred == -1, 0, 1)
pred_df = pd.DataFrame()
print(pred.shape)
print(pred)
pred_df['Bound'] = pred
pred_df.index.name = 'Id'

(3000,)
[1 1 0 ... 0 0 1]


In [26]:
pred_df.to_csv('DD_SUM_KERNEL_MISMATCH_V1.csv', sep=',', header=True)

#Public score of Sum Mismatch Kernel: 

In [25]:
pred_df.head()

,Bound
Id,
0,1
1,1
2,0
3,1
4,0
